In [1]:
from numpy import array 
from keras.preprocessing.text import Tokenizer 
from keras.utils import to_categorical 
from keras.utils.vis_utils import plot_model 
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import LSTM 
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
# generate a sequence from a language model 
def generate_seq(model, tokenizer, max_length, seed_text, n_words): 
    in_text = seed_text 
    # generate a fixed number of words 
    for _ in range(n_words): 
        # encode the text as integer 
        encoded = tokenizer.texts_to_sequences([in_text])[0] 
        # pre-pad sequences to a fixed length 
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre') 
        # predict probabilities for each word 
        yhat = model.predict_classes(encoded, verbose=0) 
        # map predicted word index to word 
        out_word = '' 
        for word, index in tokenizer.word_index.items(): 
            if index == yhat: 
                out_word = word 
                break 
        # append to input 
        in_text += ' ' + out_word 
    return in_text


In [3]:
# define the model 
def define_model(vocab_size, max_length): 
    model = Sequential() 
    model.add(Embedding(vocab_size, 10, input_length=max_length-1)) 
    model.add(LSTM(50)) 
    model.add(Dense(vocab_size, activation='softmax')) 
    # compile network 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
    # summarize defined  
    model.summary() 
    plot_model(model, to_file='word_model3_twowordsIn_onewordOut.png', show_shapes=True) 
    return model

In [4]:
# source text 
data = """ Jack and Jill went up the hill\n 
To fetch a pail of water\n Jack fell down and broke his crown\n 
And Jill came tumbling after\n """

In [5]:
# prepare the tokenizer on the source text 
tokenizer = Tokenizer() 
tokenizer.fit_on_texts([data]) 
encoded = tokenizer.texts_to_sequences([data])[0]
# determine the vocabulary size 
vocab_size = len(tokenizer.word_index) + 1 
print('Vocabulary Size: %d' % vocab_size) 

Vocabulary Size: 22


In [6]:
# encode 2 words -> 1 word 
sequences = list() 
for i in range(2, len(encoded)): 
    sequence = encoded[i-2:i+1] 
    sequences.append(sequence) 
print('Total Sequences: %d' % len(sequences))

Total Sequences: 23


In [7]:
sequences,len(sequences)

([[2, 1, 3],
  [1, 3, 4],
  [3, 4, 5],
  [4, 5, 6],
  [5, 6, 7],
  [6, 7, 8],
  [7, 8, 9],
  [8, 9, 10],
  [9, 10, 11],
  [10, 11, 12],
  [11, 12, 13],
  [12, 13, 2],
  [13, 2, 14],
  [2, 14, 15],
  [14, 15, 1],
  [15, 1, 16],
  [1, 16, 17],
  [16, 17, 18],
  [17, 18, 1],
  [18, 1, 3],
  [1, 3, 19],
  [3, 19, 20],
  [19, 20, 21]],
 23)

In [8]:
# pad input sequences 
max_length = max([len(seq) for seq in sequences]) 
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre') 
print('Max Sequence Length: %d' % max_length)


Max Sequence Length: 3


In [9]:
sequences

array([[ 2,  1,  3],
       [ 1,  3,  4],
       [ 3,  4,  5],
       [ 4,  5,  6],
       [ 5,  6,  7],
       [ 6,  7,  8],
       [ 7,  8,  9],
       [ 8,  9, 10],
       [ 9, 10, 11],
       [10, 11, 12],
       [11, 12, 13],
       [12, 13,  2],
       [13,  2, 14],
       [ 2, 14, 15],
       [14, 15,  1],
       [15,  1, 16],
       [ 1, 16, 17],
       [16, 17, 18],
       [17, 18,  1],
       [18,  1,  3],
       [ 1,  3, 19],
       [ 3, 19, 20],
       [19, 20, 21]])

In [10]:
# split into input and output elements 
sequences = array(sequences) 
X, y = sequences[:,:-1],sequences[:,-1] 
y = to_categorical(y, num_classes=vocab_size)


In [11]:
X,y

(array([[ 2,  1],
        [ 1,  3],
        [ 3,  4],
        [ 4,  5],
        [ 5,  6],
        [ 6,  7],
        [ 7,  8],
        [ 8,  9],
        [ 9, 10],
        [10, 11],
        [11, 12],
        [12, 13],
        [13,  2],
        [ 2, 14],
        [14, 15],
        [15,  1],
        [ 1, 16],
        [16, 17],
        [17, 18],
        [18,  1],
        [ 1,  3],
        [ 3, 19],
        [19, 20]]),
 array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 

In [12]:
# define model 
model = define_model(vocab_size, max_length) 







_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2, 10)             220       
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_1 (Dense)              (None, 22)                1122      
Total params: 13,542
Trainable params: 13,542
Non-trainable params: 0
_________________________________________________________________


In [13]:
# fit network 
model.fit(X, y, epochs=500, verbose=2) 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/500
 - 2s - loss: 3.0919 - acc: 0.0435
Epoch 2/500
 - 0s - loss: 3.0911 - acc: 0.0870
Epoch 3/500
 - 0s - loss: 3.0902 - acc: 0.0435
Epoch 4/500
 - 0s - loss: 3.0894 - acc: 0.0870
Epoch 5/500
 - 0s - loss: 3.0886 - acc: 0.1304
Epoch 6/500
 - 0s - loss: 3.0877 - acc: 0.1304
Epoch 7/500
 - 0s - loss: 3.0869 - acc: 0.0870
Epoch 8/500
 - 0s - loss: 3.0860 - acc: 0.0870
Epoch 9/500
 - 0s - loss: 3.0851 - acc: 0.0870
Epoch 10/500
 - 0s - loss: 3.0843 - acc: 0.0870
Epoch 11/500
 - 0s - loss: 3.0834 - acc: 0.0870
Epoch 12/500
 - 0s - loss: 3.0825 - acc: 0.0870
Epoch 13/500
 - 0s - loss: 3.0815 - acc: 0.0870
Epoch 14/500
 - 0s - loss: 3.0806 - acc: 0.0870
Epoch 15/500
 - 0s - loss: 3.0796 - acc: 0.0870
Epoch 16/500
 - 0s - loss: 3.0786 - acc: 0.0870
Epoch 17/500
 - 0s - loss: 3.0776 - acc: 0.0870
Epoch 18/500
 - 0s - loss: 3.0765 - acc: 0.0870
Epoch 19/500
 - 0s - loss: 3.0754 - acc: 0.0870
Ep

Epoch 159/500
 - 0s - loss: 1.0511 - acc: 0.9565
Epoch 160/500
 - 0s - loss: 1.0236 - acc: 0.9565
Epoch 161/500
 - 0s - loss: 0.9964 - acc: 0.9565
Epoch 162/500
 - 0s - loss: 0.9694 - acc: 0.9565
Epoch 163/500
 - 0s - loss: 0.9428 - acc: 0.9565
Epoch 164/500
 - 0s - loss: 0.9164 - acc: 0.9565
Epoch 165/500
 - 0s - loss: 0.8905 - acc: 0.9565
Epoch 166/500
 - 0s - loss: 0.8649 - acc: 0.9565
Epoch 167/500
 - 0s - loss: 0.8397 - acc: 0.9565
Epoch 168/500
 - 0s - loss: 0.8150 - acc: 0.9565
Epoch 169/500
 - 0s - loss: 0.7907 - acc: 0.9565
Epoch 170/500
 - 0s - loss: 0.7667 - acc: 0.9565
Epoch 171/500
 - 0s - loss: 0.7433 - acc: 0.9565
Epoch 172/500
 - 0s - loss: 0.7204 - acc: 0.9565
Epoch 173/500
 - 0s - loss: 0.6979 - acc: 0.9565
Epoch 174/500
 - 0s - loss: 0.6760 - acc: 0.9565
Epoch 175/500
 - 0s - loss: 0.6546 - acc: 0.9565
Epoch 176/500
 - 0s - loss: 0.6337 - acc: 0.9565
Epoch 177/500
 - 0s - loss: 0.6132 - acc: 0.9565
Epoch 178/500
 - 0s - loss: 0.5933 - acc: 0.9565
Epoch 179/500
 - 0s 

Epoch 327/500
 - 0s - loss: 0.0814 - acc: 0.9565
Epoch 328/500
 - 0s - loss: 0.0812 - acc: 0.9565
Epoch 329/500
 - 0s - loss: 0.0810 - acc: 0.9565
Epoch 330/500
 - 0s - loss: 0.0808 - acc: 0.9565
Epoch 331/500
 - 0s - loss: 0.0807 - acc: 0.9565
Epoch 332/500
 - 0s - loss: 0.0805 - acc: 0.9565
Epoch 333/500
 - 0s - loss: 0.0803 - acc: 0.9565
Epoch 334/500
 - 0s - loss: 0.0801 - acc: 0.9565
Epoch 335/500
 - 0s - loss: 0.0799 - acc: 0.9565
Epoch 336/500
 - 0s - loss: 0.0798 - acc: 0.9565
Epoch 337/500
 - 0s - loss: 0.0796 - acc: 0.9565
Epoch 338/500
 - 0s - loss: 0.0794 - acc: 0.9565
Epoch 339/500
 - 0s - loss: 0.0793 - acc: 0.9565
Epoch 340/500
 - 0s - loss: 0.0791 - acc: 0.9565
Epoch 341/500
 - 0s - loss: 0.0790 - acc: 0.9565
Epoch 342/500
 - 0s - loss: 0.0788 - acc: 0.9565
Epoch 343/500
 - 0s - loss: 0.0787 - acc: 0.9565
Epoch 344/500
 - 0s - loss: 0.0785 - acc: 0.9565
Epoch 345/500
 - 0s - loss: 0.0784 - acc: 0.9565
Epoch 346/500
 - 0s - loss: 0.0782 - acc: 0.9565
Epoch 347/500
 - 0s 

Epoch 495/500
 - 0s - loss: 0.0677 - acc: 0.9565
Epoch 496/500
 - 0s - loss: 0.0676 - acc: 0.9565
Epoch 497/500
 - 0s - loss: 0.0676 - acc: 0.9565
Epoch 498/500
 - 0s - loss: 0.0676 - acc: 0.9565
Epoch 499/500
 - 0s - loss: 0.0675 - acc: 0.9565
Epoch 500/500
 - 0s - loss: 0.0675 - acc: 0.9565


In [14]:
# evaluate model 
print(generate_seq(model, tokenizer, max_length-1, 'Jack and', 5)) 
print(generate_seq(model, tokenizer, max_length-1, 'And Jill', 3)) 
print(generate_seq(model, tokenizer, max_length-1, 'fell down', 5)) 
print(generate_seq(model, tokenizer, max_length-1, 'pail of', 5))

Jack and jill went up the hill
And Jill went up the
fell down and broke his crown and
pail of water jack fell down and
